In [0]:
#"""
!pip install nptyping
!pip install ml_metrics
!pip install hyperopt
#"""

В этом файле происходит подготовка матриц сопряженности исполнителей для проведения кластеризации.

In [0]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack, save_npz, load_npz
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import ml_metrics 
from typing import Dict, Any
from nptyping import NDArray
from sklearn.cluster import AffinityPropagation, MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import davies_bouldin_score

from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from hyperopt import hp
from matplotlib import pyplot as plt

DATA_DIR = './drive/My Drive/Colab Notebooks/VK_internship/'
sys.path.append(DATA_DIR)
recommendation = __import__("recommendation") 

Разделение на тренировочную и тестовую часть:

In [0]:
user_artists_df = pd.read_csv(DATA_DIR+"users_artists.csv", usecols=["user_id", "artists"]).drop_duplicates()
artists_id = user_artists_df["artists"].unique()
artists_id.sort()
train, test = train_test_split(artists_id, test_size=0.1)
del artists_id

Формирование матриц для обучения и тестирования:

Создание подвыборки данных и идентификаторов исполнителей, для составления строк и столбцов:

In [0]:
partial_user_artists_df = user_artists_df.loc[user_artists_df["artists"].isin(train)]
partial_artists = partial_user_artists_df.groupby("artists").count().reset_index()
partial_artists = partial_artists.sort_values(by="user_id", ascending = False)
partial_artists_rows = partial_artists.loc[:,"artists"].values
partial_artists = partial_artists_rows[:1000] 
# столбцы матриц - топ-1000 популярных исполнителей из тренировочного множества
# строки train матрицы - топ-2000 популярных исполнителей из тренировочного множества

In [5]:
partial_artists_rows.shape

(460591,)

In [0]:
partial_users = partial_user_artists_df.groupby("user_id").count().reset_index()
partial_users = partial_users.sort_values(by="artists", ascending = False).iloc[:15000]
partial_users = partial_users.loc[:,"user_id"].values
# берется информация по 5 тысячам самых активных пользователей, слушающих исполнителей из тренировочного множества
# данных очень много, чтобы обработать все не хватит памяти, 
# поэтому было принято решение брать информацию только по самым активным пользователям
del partial_user_artists_df

In [0]:
cutted_user_artists_df = user_artists_df.loc[user_artists_df["user_id"].isin(partial_users)]
# Из рассмотрения убираются избыточные пользователи
del partial_users

In [0]:
partial_user_artists_df_for_cols = cutted_user_artists_df.loc[cutted_user_artists_df["artists"].isin(partial_artists)]
# В этом датафрейме находятся исполнители из столбцов матриц train и test
# partial_user_artists_df_for_rows = cutted_user_artists_df.loc[cutted_user_artists_df["artists"].isin(partial_artists_rows)]

Формирование структуры матрицы test (координат и значений):

In [0]:
cutted_test_user_artists_df = cutted_user_artists_df.loc[cutted_user_artists_df["artists"].isin(test)] # строки матрицы test
test_df = cutted_test_user_artists_df.merge(partial_user_artists_df_for_cols, on = "user_id") 
test_df = test_df.groupby(by = ["artists_x","artists_y"]).count().reset_index() # структура матрицы test
del cutted_test_user_artists_df

Формирование sparse матрицы test:

In [0]:
le_test = LabelEncoder()
le_test = le_test.fit(test) #энкодер строк матрицы test

le_train_cols = LabelEncoder()
le_train_cols = le_train_cols.fit(partial_artists) # энкодер для столбцов матриц train и test

test_df.loc[:, "artists_x"] = le_test.transform(test_df.loc[:,"artists_x"])
test_df.loc[:, "artists_y"] = le_train_cols.transform(test_df.loc[:,"artists_y"])
test_matrix = csr_matrix((test_df["user_id"], 
                       (test_df["artists_x"], test_df["artists_y"])),
                       shape=(len(test), len(partial_artists)))
del test_df
save_npz(DATA_DIR+"test_matrix.npz", test_matrix)

Формирование структуры и sparse матрицы train:

In [0]:
batch_size = 500
batches_amount = int(len(partial_artists_rows)/batch_size)

train_matrix = csr_matrix((batch_size, len(partial_artists)))

for batch_num in range(batches_amount):
    partial_user_artists_df_for_rows = cutted_user_artists_df.loc[
        cutted_user_artists_df["artists"].isin(partial_artists_rows[batch_num*batch_size:(batch_num+1)*batch_size])
    ]
    train_df = partial_user_artists_df_for_rows.merge(partial_user_artists_df_for_cols, on="user_id") # самая дорогая по памяти операция
    del partial_user_artists_df_for_rows

    train_df = train_df.groupby(by = ["artists_x","artists_y"]).count().reset_index() # структура матрицы train

    le_train_rows = LabelEncoder()
    le_train_rows = le_train_rows.fit(train_df.loc[:,"artists_x"])

    train_df.loc[:, "artists_x"] = le_train_rows.transform(train_df.loc[:,"artists_x"])   
    train_df.loc[:, "artists_y"] = le_train_cols.transform(train_df.loc[:,"artists_y"])

    train_matrix_part = csr_matrix((train_df["user_id"], 
                          (train_df["artists_x"], train_df["artists_y"])),
                          shape=(batch_size, len(partial_artists)))
    del train_df
    if train_matrix.count_nonzero():
        train_matrix = vstack([train_matrix, train_matrix_part]) # вертикальная конкатенация sparse матриц
    else:
        train_matrix = train_matrix_part
    del train_matrix_part

del cutted_user_artists_df, partial_user_artists_df_for_cols
save_npz(DATA_DIR+"train_matrix.npz", train_matrix)